## **SETUP, INSTALLAZIONE E CONFIGURAZIONE**
Il codice è strutturato per funzionare su Colab e con l'utilizzo di Google Drive.

In [ ]:
# ==========================================
# ⚙️ CELLA 1: SETUP, INSTALLAZIONE
# ==========================================
import os
import sys
from google.colab import drive

# --- 1. MOUNT DRIVE ---
if not os.path.exists('/content/drive'):
    print("📂 Mounting Google Drive...")
    drive.mount('/content/drive')

# --- 2. INSTALLAZIONE LIBRERIE (Solo se necessario) ---
try:
    import audiocraft
    print("✅ AudioCraft già installato.")
except ImportError:
    print("⏳ Installazione dipendenze...")
    !apt-get install ffmpeg -y > /dev/null
    !pip install av einops flashy hydra-core hydra_colorlog julius num2words numpy sentencepiece spacy xformers pandas tqdm torchmetrics torchdiffeq pystoi pesq gradio demucs torchcodec
    !pip install --no-deps -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft

# --- 3. CLONE REPOSITORY ---
REPO_URL = "https://github.com/TommasoFederici/Emotion-Control-in-MusicGen-via-Steering.git"
REPO_NAME = "Emotion-Control-in-MusicGen-via-Steering"

if not os.path.exists(REPO_NAME):
    print(f"🔄 Clonazione repo: {REPO_NAME}...")
    !git clone $REPO_URL
else:
    print("🔄 Aggiornamento repo...")
    %cd $REPO_NAME
    !git pull
    %cd ..

# Aggiungi al path Python per importare le classi
if os.path.abspath(REPO_NAME) not in sys.path:
    sys.path.append(os.path.abspath(REPO_NAME))

In [ ]:
from core_colab_melody import MusicGenWrapper, DatasetExtractor, DatasetInference, Evaluation

In [ ]:
#####################################################
## -- CONFIGURAZIONE --                            ##
#####################################################

if os.path.exists(REPO_NAME):
    print(f"📂 Entro nella cartella: {REPO_NAME}")
    os.chdir(REPO_NAME)
else:
    # Se siamo già dentro (magari da esecuzioni precedenti), controlliamo
    if os.path.exists("data/melodies"):
        print("✅ Siamo già nella cartella corretta.")
    else:
        print(f"❌ ERRORE: Non trovo la cartella {REPO_NAME}")

# --- 1. CONFIGURAZIONE PARAMETRI ESPERIMENTO ---
injection_map = {
    12: list(range(11, 15)),
    27: list(range(27, 30))
}

ALPHA_CONF = {
    'mid': 1.1,   # Per il blocco 10-20
    'deep': 0.2   # Per i layer > 30
}
DECAY_FACTOR = 0.998
MIN_ALPHA = 0.2
DURATION_SEC = 5

# Generazione stringhe per nomi file/cartelle
if isinstance(ALPHA_CONF, dict):
    alpha_str = "_".join([f"{k}{str(v).replace('.', '')}" for k, v in ALPHA_CONF.items()])
else:
    alpha_str = str(ALPHA_CONF).replace('.', '_')

# layers_str_inference: Unisce tutti i layer target (es. "11_12_13_14_33_34_35")
all_targets = sorted(list(set([t for targets in injection_map.values() for t in targets])))
layers_str_inference = "_".join(map(str, all_targets))

# strategy_str: Unisce i layer sorgente (es. "14_35")
src_keys = sorted(injection_map.keys())
src_str = "_".join(map(str, src_keys))

# --- 2. CONFIGURAZIONE PERCORSI (File System) ---
# Root su Drive
DRIVE_ROOT = "/content/drive/MyDrive/AML_Project_MusicGen/MusicGen_Melody_Prova_Finale"

# Percorsi Output
VECTORS_DIR = os.path.join(DRIVE_ROOT, "vectors")
LAYER_DIR = os.path.join(DRIVE_ROOT, f"layers/layer_{layers_str_inference}_src_{src_str}_test")
TRAIN_AUDIO_DIR = os.path.join(DRIVE_ROOT, "train_audio")
TEST_AUDIO_DIR = os.path.join(LAYER_DIR, f"test_alpha_{alpha_str}")

# Percorsi Input
CSV_EXTRACTION = "data/Happy_Sad/extraction.csv"
CSV_INFERENCE = "data/Happy_Sad/inference.csv"
VEC_PATH = os.path.join(VECTORS_DIR, f"steering_vectors.pt")

# Creazione Cartelle
os.makedirs(TRAIN_AUDIO_DIR, exist_ok=True)
os.makedirs(TEST_AUDIO_DIR, exist_ok=True)
os.makedirs(VECTORS_DIR, exist_ok=True)

# --- REPORT CONFIGURAZIONE ---
print("\n🎉 SETUP COMPLETATO. Configurazione Attuale:")
print(f"🔹 Layers inference: {layers_str_inference}")
print(f"🔹 Alpha: {ALPHA_CONF}")
print(f"🔹 Output Vettore: {VEC_PATH}")
print(f"🔹 Output Audio Test: {TEST_AUDIO_DIR}")

## **ESTRAZIONE DELLO STEERING VECTOR CON ANALISI DEI LAYER DI MUSICGEN**

In [ ]:
## ESTRAZIONE E ANALISI
print(f"🏭 Estrazione Multi-Layer")

# Inizializza con modello 'melody'
mg = MusicGenWrapper(size='facebook/musicgen-melody', duration=DURATION_SEC)
extractor = DatasetExtractor(mg, layers=None)

# Esegui
extractor.run(
    csv_path=CSV_EXTRACTION,
    save_vector_path=VEC_PATH,
    output_dir_analysis=TRAIN_AUDIO_DIR,
    audio_output_dir=TRAIN_AUDIO_DIR,
    safe_zone=(10, 35),
    use_melody=False                        # IGNORA i file audio durante l'estrazione
)

## **TEST SUL DATASET DI INFERENCE**

In [ ]:
## INFERENCE

import torch
import gc

# --- PULIZIA DELLA MEMORIA (Ammazza gli Zombie) ---
print("🧹 Pulizia memoria e hook vecchi...")
if 'mg' in locals() and mg is not None:
    # Rimuoviamo gli hook se sono rimasti appesi
    for layer in mg.model.lm.transformer.layers:
        layer._forward_hooks.clear()

    # Nota: Non cancelliamo 'mg' se vogliamo riusarlo (risparmia tempo di caricamento)
    # Ma svuotiamo la cache CUDA per sicurezza
    torch.cuda.empty_cache()
    gc.collect()

# Se per caso mg non c'è (es. hai saltato la cella estrazione), lo carichiamo
if 'mg' not in locals() or mg is None:
    print("⏳ Caricamento Modello...")
    mg = MusicGenWrapper(size='facebook/musicgen-melody', duration=DURATION_SEC)

print(f"🚀 Inferenza Alpha {ALPHA_CONF} con strategia {injection_map}...")

# 1. Inizializzazione
inference = DatasetInference(mg)

# 2. Esecuzione
inference.run(
    prompts_file=CSV_INFERENCE,
    vector_path=VEC_PATH,
    output_dir=TEST_AUDIO_DIR,
    injection_map=injection_map,
    alpha=ALPHA_CONF,
    decay=DECAY_FACTOR,
    min_alpha=MIN_ALPHA,
    use_melody=False
)

print(f"✅ Tutto salvato su Drive: {TEST_AUDIO_DIR}")

## **EVALUATION DEGLI AUDIO DI EXTRACTION ED INFERENCE CON CLAP**

In [ ]:
print(f"📊 Analisi cartella: {TRAIN_AUDIO_DIR}")

Evaluation.run(
    audio_folder=TRAIN_AUDIO_DIR,
    output_dir=TRAIN_AUDIO_DIR,
    csv_filename=f"metrics_train_happiness.csv",
    train_mode=True,
    num_samples=20,
    y_label= 'Happiness Score',
    label_pos= 'Happiness vibes',
    label_neg= 'Sadness vibes'
)

print(f"📊 Analisi cartella: {TEST_AUDIO_DIR}")

Evaluation.run(
    audio_folder=TEST_AUDIO_DIR,
    output_dir=TEST_AUDIO_DIR,
    csv_filename=f"metrics_layer{layers_str_inference}_alpha_{alpha_str}.csv",
    train_mode=False,
    num_samples=50,
    y_label= 'Happiness Score',
    label_pos= 'Happiness vibes',
    label_neg= 'Sadness vibes'
)